# SCRIPT TO PERFORM QUALITY CONTROL ON WHOLE-GENOME SEQUENCING DATA

In order to run, there has to be several files in the project folder:
- GENCODE GTF: Run Scripts/WGS/01_get_gencode_annotation.sh. Obtain from: https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_46/gencode.v46.annotation.gtf.gz (Check for newer versions).

Once completed, a new Jupyter Notebook should be initialized so we can access this file. Or unmount and mount again the project (https://community.ukbiobank.ac.uk/hc/en-gb/community/posts/16019592366365-It-seems-that-the-recently-dx-uploaded-files-does-not-show-up-on-mnt-project-until-I-re-start-the-whole-Jupyter-Lab-VM)


- PVCF BLOCKS: Run Notebooks/WGS/DragenBlockProcessing.ipynb. Obtain from: https://biobank.ndph.ox.ac.uk/ukb/ukb/auxdata/dragen_pvcf_coordinates.zip 
It needs parsing, but in data/misc it is already parsed.

- Samples to remove file: Run Notebooks/WGS/01_QC_Samples.ipynb

#### Initialization 
##### Load packages


In [1]:
import dxpy
import pyspark

import hail as hl
from pathlib import Path
from datetime import datetime

from src.matrixtables import smart_split_multi_mt

In [2]:
# Constants
DATABASE = "matrix_tables"
REFERENCE_GENOME = "GRCh38"
PROJ_NAME = "OPRM1"

Path("/tmp").resolve().mkdir(parents=True, exist_ok=True)

LOG_FILE = (
    Path("../hail_logs", f"{PROJ_NAME}_{datetime.now().strftime('%H%M')}.log")
    .resolve()
    .__str__()
)

#### Hail and spark configuration

In [3]:
# Spark init
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

# Create database in DNAX
spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE} LOCATION 'dnax://'")
mt_database = dxpy.find_one_data_object(name=DATABASE, classname="database")["id"]

# Hail init
hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.2.3
SparkUI available at http://ip-10-60-140-241.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.116-cd64e0876c94
LOGGING: writing to /opt/hail_logs/OPRM1_1520.log


#### Variables

In [4]:
# RAP
VCF_VERSION = "v1"
FIELD_ID = 24310 # DRAGEN population level WGS variants, pVCF format 500k release

# Paths
BULK_DIR = Path("/mnt/project/Bulk")

# Genes
GENES = ["OPRM1"]

### Quality control

#### Gene intervals and blocks 

In [5]:
# Get gene intervals
gene_interval = hl.experimental.get_gene_intervals(
    gene_symbols=GENES,
    reference_genome="GRCh38",
    gtf_file="file:///mnt/project/WGS_Lucia/WGS_QC/gencode.v46.annotation.gtf",
)
gene_interval

2025-01-08 15:21:41.562 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type int32 (user-supplied)
  Loading field 'f4' as type int32 (user-supplied)
  Loading field 'f5' as type float64 (user-supplied)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type int32 (user-supplied)
  Loading field 'f8' as type str (not specified)
2025-01-08 15:22:00.737 Hail: INFO: wrote table with 3467156 rows in 12 partitions to /tmp/ekMXe5LAVfVL1qFYZAVpCV
2025-01-08 15:22:05.077 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-01-08 15:22:09.965 Hail: INFO: get_gene_intervals found 1 entries:
gene: OPRM1 (ENSG00000112038)


[Interval(start=Locus(contig=chr6, position=154010496, reference_genome=GRCh38), end=Locus(contig=chr6, position=154246867, reference_genome=GRCh38), includes_start=True, includes_end=True)]

In [6]:
# Get DRAGEN pVCF blocks
blocks = hl.import_table("file:///mnt/project/WGS_Lucia/WGS_QC/dragen_pvcf_blocks.tsv", no_header=False)
blocks = blocks.annotate(Chromosome=blocks.Chromosome.replace("23", "X").replace("24", "Y"))
blocks = blocks.annotate(region=hl.str("").join([hl.str("chr"), blocks.Chromosome]))
blocks = blocks.annotate(
    interval=hl.locus_interval(
        blocks.region,
        hl.int32(blocks.Starting_Position),
        hl.int32(blocks.Ending_Position),
        reference_genome="GRCh38",
    )
).key_by("interval")

2025-01-08 15:22:11.304 Hail: INFO: Reading table without type imputation
  Loading field 'Row_Number' as type str (not specified)
  Loading field 'Chromosome' as type str (not specified)
  Loading field 'Block' as type str (not specified)
  Loading field 'Starting_Position' as type str (not specified)
  Loading field 'Ending_Position' as type str (not specified)


In [7]:
# Get blocks for given genes
gb = blocks.filter(hl.any(lambda inter: blocks.interval.overlaps(inter), gene_interval))
gb.show()

,,,,,,
Row_Number,Chromosome,Block,Starting_Position,Ending_Position,region,interval
str,str,str,str,str,str,interval<locus<GRCh38>>
"""60766""","""6""","""7700""","""153991433""","""154011429""","""chr6""",[chr6:153991433-chr6:154011429)
"""60767""","""6""","""7701""","""154011430""","""154031420""","""chr6""",[chr6:154011430-chr6:154031420)
"""60768""","""6""","""7702""","""154031421""","""154051414""","""chr6""",[chr6:154031421-chr6:154051414)
"""60769""","""6""","""7703""","""154051415""","""154071408""","""chr6""",[chr6:154051415-chr6:154071408)
"""60770""","""6""","""7704""","""154071409""","""154091406""","""chr6""",[chr6:154071409-chr6:154091406)
"""60771""","""6""","""7705""","""154091407""","""154111396""","""chr6""",[chr6:154091407-chr6:154111396)
"""60772""","""6""","""7706""","""154111397""","""154131395""","""chr6""",[chr6:154111397-chr6:154131395)
"""60773""","""6""","""7707""","""154131396""","""154151386""","""chr6""",[chr6:154131396-chr6:154151386)


#### Import vcf files of specific blocks

In [8]:
VCF_DIR = Path("DRAGEN WGS/DRAGEN population level WGS variants, pVCF format 500k release")

vcf_files = [
    f"file://{BULK_DIR / VCF_DIR}/{chromosome}/ukb{FIELD_ID}_c{chromosome.replace('chr', '')}_b{block}_{VCF_VERSION}.vcf.gz"
    for block, chromosome in zip(gb.Block.collect(), gb.region.collect())
]

mt = hl.import_vcf(
    vcf_files,
    drop_samples=False,
    reference_genome="GRCh38",
    array_elements_required=False,
    force_bgz=True,
)

2025-01-08 15:22:15.386 Hail: INFO: Coerced sorted dataset
2025-01-08 15:22:17.755 Hail: INFO: Coerced sorted dataset


In [9]:
# Only genes of interest
mt = hl.filter_intervals(mt, gene_interval)
print(f"{mt.count_rows()} variants after gene filtering")

2025-01-08 15:22:44.497 Hail: INFO: scanning VCF for sortedness...
2025-01-08 15:24:43.920 Hail: INFO: Coerced sorted VCF - no additional import work to do


78512 variants after gene filtering


In [10]:
# Remove singletons (variants that appear only once across all samples)
mt = mt.filter_rows(hl.agg.sum(mt.GT.n_alt_alleles()) > 1)
print(f"{mt.count_rows()} variants after removing singletons")

46265 variants after removing singletons


In [11]:
# First checkpoint
stage = "FIRST"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 15:35:54.977 Hail: INFO: wrote matrix table with 46265 rows and 490541 columns in 678 partitions to /tmp/OPRM1.FIRST.cp.mt


#### Multi-allele filtering

In [12]:
# Remove variants with 6 or more alleles
mt = mt.filter_rows(mt.alleles.length() <= 6)
print(f"{mt.count_rows()} variants with not more than 6 alleles")

46054 variants with not more than 6 alleles


In [13]:
# Split multi-allele variants into single ones
mt = smart_split_multi_mt(mt)
print(f"{mt.count_rows()} variants after multi-allele splitting")

59311 variants after multi-allele splitting


In [14]:
# Second checkpoint
stage = "SECOND"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 15:47:17.544 Hail: INFO: wrote matrix table with 59311 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.SECOND.cp.mt


#### Quality control filtering

In [15]:
mt = mt.filter_entries(mt.FT == "PASS")

# Then, filter variants where there is at least one non-missing genotype
mt = mt.filter_rows(hl.agg.any(hl.is_defined(mt.GT)))

In [16]:
# third checkpoint
stage = "THIRD"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 15:49:50.784 Hail: INFO: wrote matrix table with 59311 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.THIRD.cp.mt


In [17]:
# Compute statistics about the number and fraction of filtered entries.
mt = hl.MatrixTable.compute_entry_filter_stats(mt, row_field='entry_stats_row', col_field='entry_stats_col')

In [18]:
# forth checkpoint
stage = "FORTH"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 15:57:23.185 Hail: INFO: wrote matrix table with 59311 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.FORTH.cp.mt


In [19]:
row_fraction_threshold = 0.95

# Filter variants where at least 95% of genotypes are unfiltered
mt = mt.filter_rows(
    (1 - mt.entry_stats_row.fraction_filtered) > row_fraction_threshold
)


In [20]:
col_fraction_threshold = 0.95

# Filter samples where at least 95% of variants are unfiltered
mt = mt.filter_cols(
    (1 - mt.entry_stats_col.fraction_filtered) > col_fraction_threshold
)


In [21]:
# Five checkpoint
stage = "FIVE"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 16:01:25.903 Hail: INFO: wrote matrix table with 58863 rows and 490541 columns in 1356 partitions to /tmp/OPRM1.FIVE.cp.mt


#### Remove samples from 01_QC_Samples.ipynb

In [22]:
samples_to_remove = hl.import_table("file:///mnt/project/WGS_Lucia/Data/samples_to_remove.tsv", key="eid")

mt = mt.anti_join_cols(samples_to_remove)

# Filter rows (variants) where any sample information is still present
mt = mt.filter_rows(hl.agg.any(mt.GT.n_alt_alleles() > 0))


2025-01-08 16:01:58.578 Hail: INFO: Reading table without type imputation
  Loading field 'eid' as type str (not specified)


In [23]:
# Six checkpoint
stage = "SIX"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 16:05:50.241 Hail: INFO: wrote matrix table with 52078 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.SIX.cp.mt


#### Variant Effect Predictor (VEP)

In [24]:
VEP_JSON = Path("GRCh38_VEP.json").resolve()

In [25]:
mt = hl.vep(mt, f"file:{VEP_JSON}", block_size = 100)

2025-01-08 16:08:19.820 Hail: INFO: wrote table with 52078 rows in 1356 partitions to /tmp/persist_TablekTkRfP55gd


In [26]:
is_MANE = mt.aggregate_rows(
    hl.agg.all(hl.is_defined(mt.vep.transcript_consequences.mane_select))
)
assert is_MANE, "Selected transcript may not be MANE Select. Check manually."

mt = mt.annotate_rows(
    protCons=mt.vep.transcript_consequences.amino_acids[0].split("/")[0]
    + hl.str(mt.vep.transcript_consequences.protein_end[0])
    + mt.vep.transcript_consequences.amino_acids[0].split("/")[-1],
    varid=hl.variant_str(mt.locus, mt.alleles)
)

In [27]:
# Seven checkpoint
stage = "SEVEN"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 16:15:15.568 Hail: INFO: wrote matrix table with 52078 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.SEVEN.cp.mt


### Filtering

In [28]:
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.n_non_ref > 0)
mt = mt.filter_rows(mt.variant_qc.gq_stats.mean >= 20)
mt = mt.filter_rows(mt.variant_qc.call_rate >= 0.95)
mt = mt.filter_rows(mt.vep.most_severe_consequence != "intron_variant")
mt = mt.filter_rows(mt.vep.most_severe_consequence != "downstream_gene_variant")
mt = mt.filter_rows(mt.vep.most_severe_consequence != "upstream_gene_variant")

In [29]:
print(f"{mt.count_rows()} variants after quality filtering")

4756 variants after quality filtering


In [30]:
# Eight checkpoint
stage = "EIGHT"
checkpoint_file = f"/tmp/{PROJ_NAME}.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

2025-01-08 16:21:15.286 Hail: INFO: wrote matrix table with 4756 rows and 431455 columns in 1356 partitions to /tmp/OPRM1.EIGHT.cp.mt


### Formating

In [31]:
qt = mt.rows()
qt = qt.explode(qt.vep.transcript_consequences)

qt = qt.select(
    qt.varid,
    qt.protCons,
    qt.vep.most_severe_consequence,
    qt.vep.transcript_consequences.protein_end,
    qt.vep.transcript_consequences.protein_start,
    qt.vep.transcript_consequences.amino_acids,
    qt.vep.transcript_consequences.gene_id,
    qt.vep.transcript_consequences.transcript_id,
    **qt.variant_qc.flatten(),
)

qt = qt.annotate(AC=qt.AC[1], AF=qt.AF[1], homozygote_count=qt.homozygote_count[1])
qt = qt.key_by().drop("locus", "alleles")

qt.show(5)

,,,,,,,,,,,,,,,,,,,,,,,,
varid,protCons,most_severe_consequence,protein_end,protein_start,amino_acids,gene_id,transcript_id,gq_stats.mean,gq_stats.stdev,gq_stats.min,gq_stats.max,AC,AF,AN,homozygote_count,call_rate,n_called,n_not_called,n_filtered,n_het,n_non_ref,het_freq_hwe,p_value_hwe,p_value_excess_het
str,str,str,int32,int32,str,str,str,float64,float64,float64,float64,int32,float64,int32,int32,float64,int64,int64,int64,int64,int64,float64,float64,float64
"""chr6:154010515:G:C""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,1,1.16e-06,862910,0,1.00e+00,431455,0,0,1,1,2.32e-06,5.00e-01,5.00e-01
"""chr6:154010520:G:A""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,2,2.32e-06,862908,0,1.00e+00,431454,0,1,2,2,4.64e-06,5.00e-01,5.00e-01
"""chr6:154010521:G:A""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,3,3.48e-06,862908,0,1.00e+00,431454,0,1,3,3,6.95e-06,5.00e-01,5.00e-01
"""chr6:154010530:C:T""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,3,3.48e-06,862910,0,1.00e+00,431455,0,0,3,3,6.95e-06,5.00e-01,5.00e-01
"""chr6:154010543:G:T""",NA,"""5_prime_UTR_variant""",NA,NA,NA,"""ENSG00000112038""","""ENST00000434900""",5.75e+01,1.38e+01,1.30e+01,9.90e+01,8,9.27e-06,862910,0,1.00e+00,431455,0,0,8,8,1.85e-05,5.00e-01,5.00e-01


In [32]:
# Group by each distinct 'most_severe_consequence' and count the number of occurrences
consequence_counts = qt.aggregate(
    hl.agg.group_by(qt.most_severe_consequence, hl.agg.count())
)

print(consequence_counts)

{'3_prime_UTR_variant': 3960, '5_prime_UTR_variant': 221, 'frameshift_variant': 14, 'inframe_deletion': 3, 'inframe_insertion': 2, 'missense_variant': 339, 'splice_donor_5th_base_variant': 3, 'splice_donor_region_variant': 6, 'splice_donor_variant': 5, 'splice_polypyrimidine_tract_variant': 32, 'splice_region_variant': 16, 'stop_gained': 22, 'synonymous_variant': 133}


#### Export 

In [33]:
qt.export("/tmp/variant_qc.tsv")
!hadoop fs -getmerge /tmp/variant_qc.tsv ../variant_qc.tsv
!dx upload ../variant_qc.tsv --path /WGS_Lucia/WGS_QC/Output/

2025-01-08 16:21:57.003 Hail: INFO: merging 1357 files totalling 975.2K...
2025-01-08 16:21:57.337 Hail: INFO: while writing:
    /tmp/variant_qc.tsv
  merge time: 332.957ms


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cluster/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cluster/dnax/jars/dnanexus-api-0.1.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
2025-01-08 16:22:28,722 WARN metrics.MetricsReporter: Unable to initialize metrics scraping configurations from hive-site.xml. Message:InputStream cannot be null
2025-01-08 16:22:28,894 WARN service.DNAxApiSvc: Using default configurations. Unable to find dnanexus.conf.location=null
2025-01-08 16:22:28,895 INFO service.DNAxApiSvc: apiserver connection-pool config. MaxPoolSize=10, MaxPoolPerRoute=10,MaxWaitTimeout=60000
2025-01-08 16:22:28,895 INFO service.DNAxApiSvc: initializing http connection man

In [34]:
# BGEN file
BGEN_FILE = "/tmp/OPRM1"
GPs = hl.literal([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])

mt = mt.annotate_entries(GP=GPs[mt.GT.n_alt_alleles()])

hl.export_bgen(
    mt=mt, varid=mt.varid, rsid=mt.varid, gp=mt.GP, output="file:" + BGEN_FILE
)

In [35]:
# ANNOTATIONS file
ANNOTATIONS_FILE = "/tmp/OPRM1.annotations"

annotations = (
    mt.select_rows(
        varid=mt.varid,
        gene=mt.vep.transcript_consequences.gene_symbol[0],
        annotation=hl.if_else(
            # Check if 'protCons' is missing, if so, use "most_severe_consequence"
            hl.is_missing(mt.protCons),  
            mt.vep.most_severe_consequence,  
            mt.protCons 
        )
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)

annotations.export("file:" + ANNOTATIONS_FILE, header=False)

2025-01-08 16:22:53.793 Hail: INFO: Coerced sorted dataset
2025-01-08 16:23:00.693 Hail: INFO: merging 84 files totalling 205.0K...
2025-01-08 16:23:00.789 Hail: INFO: while writing:
    file:/tmp/OPRM1.annotations
  merge time: 95.718ms


In [36]:
ANNOTATIONS_FILE_2 = "/tmp/OPRM1.annotations_most_severe"

# Process the annotations to always include "most_severe_consequence"
annotations_most_severe = (
    mt.select_rows(
        varid=mt.varid,
        gene=mt.vep.transcript_consequences.gene_symbol[0],
        annotation=mt.vep.most_severe_consequence  # Always use most severe consequence
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)

# Export the processed annotations to a second file
annotations_most_severe.export("file:" + ANNOTATIONS_FILE_2, header=False)

2025-01-08 16:23:05.018 Hail: INFO: Coerced sorted dataset
2025-01-08 16:23:07.895 Hail: INFO: merging 84 files totalling 210.8K...
2025-01-08 16:23:08.004 Hail: INFO: while writing:
    file:/tmp/OPRM1.annotations_most_severe
  merge time: 108.579ms


In [37]:
# SETLIST file
SETLIST_FILE = "/tmp/OPRM1.setlist"
position = mt.aggregate_rows(hl.agg.min(mt.locus.position))
names = mt.varid.collect()
names_str = ",".join(names)

line = f"{mt.vep.transcript_consequences.gene_symbol.collect()[0]}\t{mt.locus.contig.collect()[0]}\t{position}\t{names_str}"

with open(SETLIST_FILE, "w") as f:
    f.write(line)

In [38]:
bgen_file = BGEN_FILE + ".bgen"
sample_file = BGEN_FILE + ".sample"

!dx upload $bgen_file $sample_file $ANNOTATIONS_FILE $SETLIST_FILE --path /WGS_Lucia/WGS_QC/Output/

[===========================================================>] Uploaded 19,191,086 of 19,191,086 bytes (100%) /tmp/OPRM1.bgen
ID                                file-GxzBGxQJb4JG57Qf53j077fB
Class                             file
Project                           project-GfVK998Jb4JJgVBjKXPyxJ9q
Folder                            /WGS_Lucia/WGS_QC/Output
Name                              OPRM1.bgen
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Jan  8 16:23:18 2025
Created by                        luciass6
 via the job                      job-Gxz94k0Jb4J0gPK0F16KqkJx
Last modified                     Wed Jan  8 16:23:19 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-dnanexus"
[=============

In [39]:
!dx upload $ANNOTATIONS_FILE_2 --path /WGS_Lucia/WGS_QC/Output/

[===========================================================>] Uploaded 215,861 of 215,861 bytes (100%) /tmp/OPRM1.annotations_most_severe
ID                                file-GxzBK1jJb4J37BPq6VvKX5z6
Class                             file
Project                           project-GfVK998Jb4JJgVBjKXPyxJ9q
Folder                            /WGS_Lucia/WGS_QC/Output
Name                              OPRM1.annotations_most_severe
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Wed Jan  8 16:25:43 2025
Created by                        luciass6
 via the job                      job-Gxz94k0Jb4J0gPK0F16KqkJx
Last modified                     Wed Jan  8 16:25:44 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloud